In [14]:
import pandas as pd
import numpy as np
import keras
import tensorflow as tf
from keras.preprocessing.sequence import TimeseriesGenerator
import plotly.graph_objs as go
from keras.layers import Dropout

In [15]:
filename = "AOA.csv"
df = pd.read_csv(filename)
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1258 entries, 0 to 1257
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       1258 non-null   object 
 1   Open       1258 non-null   float64
 2   High       1258 non-null   float64
 3   Low        1258 non-null   float64
 4   Close      1258 non-null   float64
 5   Adj Close  1258 non-null   float64
 6   Volume     1258 non-null   int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 68.9+ KB
None


In [16]:
df['Date'] = pd.to_datetime(df['Date'])
df.set_axis(df['Date'], inplace=True)
df.drop(columns=['Open', 'High', 'Low', 'Volume'], inplace=True)

In [17]:
close_data = df['Close'].values
close_data = close_data.reshape((-1,1))

split_percent = 0.80
split = int(split_percent*len(close_data))

close_train = close_data[:split]
close_test = close_data[split:]

date_train = df['Date'][:split]
date_test = df['Date'][split:]

print(len(close_train))
print(len(close_test))

1006
252


In [18]:
look_back = 10

train_generator = TimeseriesGenerator(close_train, close_train, length=look_back, batch_size=20)     
test_generator = TimeseriesGenerator(close_test, close_test, length=look_back, batch_size=1)


In [33]:
from keras.models import Sequential
from keras.layers import LSTM, Dense

model = Sequential()
model.add(
    LSTM(units = 50,return_sequences = True,
        activation='relu',
        input_shape=(look_back,1))
)
model.add(
    LSTM(50,
        activation='relu',
        input_shape=(look_back,1), return_sequences = True)
)

model.add(
    LSTM(50,
        activation='relu',
        input_shape=(look_back,1), return_sequences = True)
)


model.add(LSTM(50, activation = "relu", input_shape=(look_back,1)))

model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

num_epochs = 50
model.fit(train_generator, epochs=num_epochs, verbose=1)

Epoch 1/50
50/50 [==============================] - 4s 10ms/step - loss: 465.8216
Epoch 2/50
50/50 [==============================] - 1s 10ms/step - loss: 2.9110
Epoch 3/50
50/50 [==============================] - 1s 10ms/step - loss: 3.3669
Epoch 4/50
50/50 [==============================] - 1s 10ms/step - loss: 3.4767
Epoch 5/50
50/50 [==============================] - 1s 10ms/step - loss: 2.6749
Epoch 6/50
50/50 [==============================] - 1s 11ms/step - loss: 3.1063
Epoch 7/50
50/50 [==============================] - 1s 11ms/step - loss: 3.2998
Epoch 8/50
50/50 [==============================] - 1s 10ms/step - loss: 2.1101
Epoch 9/50
50/50 [==============================] - 1s 11ms/step - loss: 2.2029
Epoch 10/50
50/50 [==============================] - 1s 12ms/step - loss: 2.7689
Epoch 11/50
50/50 [==============================] - 1s 13ms/step - loss: 4.8769
Epoch 12/50
50/50 [==============================] - 1s 15ms/step - loss: 4.1882
Epoch 13/50
50/50 [================

In [34]:
prediction = model.predict(test_generator)

close_train = close_train.reshape((-1))
close_test = close_test.reshape((-1))
prediction = prediction.reshape((-1))

trace1 = go.Scatter(
    x = date_train,
    y = close_train,
    mode = 'lines',
    name = 'Data'
)
trace2 = go.Scatter(
    x = date_test,
    y = prediction,
    mode = 'lines',
    name = 'Prediction'
)
trace3 = go.Scatter(
    x = date_test,
    y = close_test,
    mode='lines',
    name = 'Ground Truth'
)
layout = go.Layout(
    title = "Google Stock",
    xaxis = {'title' : "Date"},
    yaxis = {'title' : "Close"}
)
fig = go.Figure(data=[trace1, trace2, trace3], layout=layout)
fig.show()

In [35]:
close_data = close_data.reshape((-1))
look_back = 10
def predict(num_prediction, model):
    prediction_list = close_data[-look_back:]
    
    for _ in range(num_prediction):
        x = prediction_list[-look_back:]
        x = x.reshape((1, look_back, 1))
        out = model.predict(x)[0][0]
        prediction_list = np.append(prediction_list, out)
    prediction_list = prediction_list[look_back-1:]
        
    return prediction_list
    
def predict_dates(num_prediction):
    last_date = df['Date'].values[-1]
    prediction_dates = pd.date_range(last_date, periods=num_prediction+1).tolist()
    return prediction_dates

num_prediction = 30
forecast = predict(num_prediction, model)
forecast_dates = predict_dates(num_prediction)


In [36]:
close_train = close_train.reshape((-1))
close_test = close_test.reshape((-1))
prediction = forecast.reshape((-1))

trace1 = go.Scatter(
    x = forecast_dates,
    y = close_train,
    mode = 'lines',
    name = 'Data'
)
trace2 = go.Scatter(
    x = forecast_dates,
    y = prediction,
    mode = 'lines',
    name = 'Prediction'
)
trace3 = go.Scatter(
    x = date_test,
    y = close_test,
    mode='lines',
    name = 'Ground Truth'
)
layout = go.Layout(
    title = "Google Stock",
    xaxis = {'title' : "Date"},
    yaxis = {'title' : "Close"}
)
fig = go.Figure(data=[trace2, trace3], layout=layout)
fig.show()

## ADJUSTED THIS IS FOR PORTFOLIO OPTIMIZATION

In [37]:
adj_close_data = df["Adj Close"].values
adj_close_data = adj_close_data.reshape((-1))
look_back = 10
def predict(num_prediction, model):
    prediction_list = adj_close_data[-look_back:]
    
    for _ in range(num_prediction):
        x = prediction_list[-look_back:]
        x = x.reshape((1, look_back, 1))
        out = model.predict(x)[0][0]
        prediction_list = np.append(prediction_list, out)
    prediction_list = prediction_list[look_back-1:]
        
    return prediction_list
    
def predict_dates(num_prediction):
    last_date = df['Date'].values[-1]
    prediction_dates = pd.date_range(last_date, periods=num_prediction+1).tolist()
    return prediction_dates

num_prediction = 30
adj_forecast = predict(num_prediction, model)
forecast_dates = predict_dates(num_prediction)



#--------------------------

adj_close_train = adj_close_data[:split]
adj_close_test = adj_close_data[split:]

adj_close_train = adj_close_train.reshape((-1))
adj_close_test = adj_close_test.reshape((-1))
adj_prediction = adj_forecast.reshape((-1))

trace1 = go.Scatter(
    x = forecast_dates,
    y = adj_close_train,
    mode = 'lines',
    name = 'Data'
)
trace2 = go.Scatter(
    x = forecast_dates,
    y = adj_prediction,
    mode = 'lines',
    name = 'Prediction'
)
trace3 = go.Scatter(
    x = date_test,
    y = adj_close_test,
    mode='lines',
    name = 'Ground Truth'
)
layout = go.Layout(
    title = "Google Stock",
    xaxis = {'title' : "Date"},
    yaxis = {'title' : "Close"}
)
fig = go.Figure(data=[ trace2, trace3], layout=layout)
fig.show()

## ADDING ALL INFO INTO A FILE

In [38]:
#get all close prices values with predicted values (for protfolio prediction)
import datetime

indexes = pd.date_range(start='31/12/2021', end='1/30/2022')

newdf = pd.DataFrame(indexes, columns = ["Date"])
newdf["Close"] = forecast
newdf["Adj Close"] = adj_forecast
dffinal = df.loc[:,["Date" ,"Close","Adj Close"]]
dffinal = dffinal.append(newdf)
dffinal['Date'] = pd.to_datetime(dffinal['Date'])
dffinal

,Date,Close,Adj Close
2017-01-03 00:00:00,2017-01-03,47.439999,41.429417
2017-01-04 00:00:00,2017-01-04,47.820000,41.761269
2017-01-05 00:00:00,2017-01-05,48.000000,41.918465
2017-01-06 00:00:00,2017-01-06,47.939999,41.866070
2017-01-09 00:00:00,2017-01-09,47.799999,41.743805
...,...,...,...
26,2022-01-26,70.651749,70.467628
27,2022-01-27,70.581795,70.393631
28,2022-01-28,70.493065,70.295830
29,2022-01-29,70.401657,70.193893


In [39]:
dffinal.to_csv("AOApredict.csv", index = False)

## PORTFOLIO OPTIMIZATION STATS

In [ ]:
dffinal = pd.read_csv("AOApredict.csv")

In [ ]:
dffinal.set_axis(dffinal['Date'], inplace=True)
dffinal.drop(columns=["Date"],inplace = True)

In [ ]:
#this are the predicted valyes 

dffinal.tail(60)